In [1]:
import tensorflow as tf

In [2]:
node1 = tf.constant(3.0,tf.float32)
node2 = tf.constant(4.0,tf.float32)
print(node1, node2)

(<tf.Tensor 'Const:0' shape=() dtype=float32>, <tf.Tensor 'Const_1:0' shape=() dtype=float32>)


In [3]:
sess = tf.Session()
print(sess.run([node1,node2]))

[3.0, 4.0]


In [4]:
node3 = tf.add(node1, node2)
print('node3 :', node3)
print(sess.run(node3))

('node3 :', <tf.Tensor 'Add:0' shape=() dtype=float32>)
7.0


In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b #+ provides a shortcut for tf.add
print(sess.run(adder_node,{a : 3, b: 4}))
print(sess.run(adder_node,{a : [0,1], b: [3,4]}))

7.0
[ 3.  5.]


In [6]:
# can make it slightly more complex
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a : 3, b : 4}))
print(sess.run(add_and_triple, {a  : [0,1], b: [3,4]}))

21.0
[  9.  15.]


In [7]:
# Variables
W = tf.Variable([.3],tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
lin_reg = W*x + b

In [8]:
init = tf.global_variables_initializer()
sess.run(init)

In [9]:
print(sess.run(lin_reg, {x : [1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [10]:
y = tf.placeholder(tf.float32)
sq_deltas = tf.square(lin_reg - y)
loss = tf.reduce_sum(sq_deltas)
print(sess.run(loss,{x: [1,2,3,4], y: [0,-1,-2,-3]}))

23.66


In [11]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


In [19]:
# tf.train API
optimizer = tf.train.GradientDescentOptimizer(0.01)#takes input lr
train = optimizer.minimize(loss)
sess.run(init) #reset all variables to initial values
for i in range(1000):
    sess.run(train,{x : [1,2,3,4], y : [0,-1,-2,-3]})
print (sess.run([W,b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]
